In [ ]:
!pip install transformers

In [ ]:
import os, copy
import pandas as pd
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
import warnings
from google.colab import drive

warnings.filterwarnings('ignore')
drive.mount("/content/Drive/", force_remount=True)

Mounted at /content/Drive/


In [ ]:
origin_df = pd.read_csv('/content/Drive/MyDrive/NP_classifier/mod_intent_model_dataset.csv')
df = copy.deepcopy(origin_df)
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.drop(df.loc[df['sentence'].str.contains('\*')].index, axis=0, inplace=True)
df = df.loc[df['sentence'].str.len() <= 100]
df.reset_index(inplace=True, drop=True)


df.loc[(df['sentence'].str.contains('응') & df['sentence'].str.contains('좋아')), 'intent'] = '긍정'
df.loc[(df['sentence'].str.contains('응') & df['sentence'].str.contains('맞아')), 'intent'] = '긍정'

df.loc[(df['sentence'].str.contains('아니') & df['sentence'].str.contains('싫어')), 'intent'] = '부정'


fst_P_key = ['맞어','맞아','마자','마쟈','마저','맞지','응 ','응!','응응 ','웅 ','웅웅 ','웅!','좋아 ','좋다']
fst_N_key = ['아뇨','아니','아닌데','아녀','아니여','아냐','싫어 ','싫어,','싫어!','시러','싫은데','안돼','안되지','안된','별로야']

for i in fst_P_key:
    df.loc[(df['sentence'].str.startswith(i)), 'intent'] = '긍정'
for i in fst_N_key:
    df.loc[(df['sentence'].str.startswith(i)), 'intent'] = '부정'

df

,sentence,intent
0,애덜앙 나 너무 배불러서 배 아파,(단언) 진술하기
1,배 아프면 약국 가서 소화제라도 마시는 거 어때,(지시) 충고/제안하기
2,떡볶이랑 튀김이랑 닭껍질튀김이랑 마카롱 먹었어,(단언) 진술하기
3,아닌데 그냥 많이 먹은 거 같은데,부정
4,그냥 나 뭐 사주기 싫어서 그런 거 같은데.,(단언) 주장하기
...,...,...
1142022,매물 위치와 평수를 알 수 있을까요?,(지시) 질문하기
1142023,잘 진행되고 있나요?,(지시) 질문하기
1142024,"네, 안녕하세요?",(표현) 인사하기
1142025,네~ 안녕하세요,(표현) 인사하기


In [ ]:
asdf = list(df['intent'].unique())
for i in asdf :
    print(i,df.loc[df['intent']==i,'intent'].count())

(단언) 진술하기 308107
(지시) 충고/제안하기 32575
부정 18653
(단언) 주장하기 402871
(지시) 질문하기 267339
(지시) 부탁하기 4733
(단언) 반박하기 2758
(표현) 감사하기 1506
(표현) 사과하기 532
긍정 70620
(지시) 명령/요구하기 3647
턴토크 사인(관습적 반응) 3834
(언약) 약속하기(제3자와)/(개인적 수준) 13057
(표현) 긍정감정 표현하기 4998
(표현) 부정감정 표현하기 5867
(언약) 거절하기 372
(선언/위임하기) 97
(표현) 인사하기 461


In [ ]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

In [ ]:
# token 바구니
N_count = []

# 필요 문장 추출
N_intent = ['(표현) 부정감정 표현하기', '(단언) 반박하기', '(언약) 거절하기', '부정']

# 필요 문장 구성하는 모든 token 바구니에 넣기
for intent in N_intent :
    for i in df.loc[(df['intent'] == intent), 'sentence']:
        N_count += tokenizer.encode(i)

# token 고유값과 개수 / N_uni[0] = 고유값 리스트 / N_uni[1] = 고유값 개수 리스트
N_uni = np.unique(N_count, return_counts=True)

# 고유값 개수 내림차순 정렬
N_cnt = sorted(list(set(N_uni[1])),reverse=True)

# N_many = N_uni[0]에서 검색할 index값 / N_token = 사용 횟수가 많은 token
N_many = []
N_token = []

for i in N_cnt:
    if i > 2700 :
        N_many.append(list(N_uni[1]).index(i))
for j in range(len(N_many)) :
    N_token.append(N_uni[0][N_many[j]])
print(N_many)

tokenizer.decode(N_token)

[1, 2209, 13, 1339, 1181, 950, 1145, 1144, 28, 1180, 1203, 1206, 662, 1166, 1255, 1205]


'[CLS] 아니.는키 키이어?고가도 안은에지'

In [ ]:
# token 바구니
P_count = []

# 필요 문장 추출
P_intent = ['(표현) 긍정감정 표현하기', '(표현) 감사하기', '긍정']

# 필요 문장 구성하는 모든 token 바구니에 넣기
for intent in P_intent :
    for i in df.loc[(df['intent'] == intent), 'sentence']:
        P_count += tokenizer.encode(i)

# token 고유값과 개수 / N_uni[0] = 고유값 리스트 / N_uni[1] = 고유값 개수 리스트
P_uni = np.unique(P_count, return_counts=True)

# 고유값 개수 내림차순 정렬
P_cnt = sorted(list(set(P_uni[1])),reverse=True)

# N_many = N_uni[0]에서 검색할 index값 / N_token = 사용 횟수가 많은 token
P_many = []
P_token = []

for i in P_cnt:
    if i > 7700 :
        P_many.append(list(P_uni[1]).index(i))
for j in range(len(P_many)) :
    P_token.append(P_uni[0][P_many[j]])
print(P_many)

tokenizer.decode(P_token)

[1, 851, 12010, 13, 1327, 1067, 1489, 1290, 1356, 1291, 1326, 1405, 1355, 1353, 1312, 862, 16218, 245]


'[CLS] 응 맞아.키 키는어도이고에지가은 있 응응 나'

In [ ]:
# token 바구니
M_count = []

# 필요 문장 추출
M_intent = ['(단언) 진술하기', '(지시) 충고/제안하기', '(단언) 주장하기', '(지시) 질문하기', '(지시) 부탁하기',
        '(표현) 사과하기', '(지시) 명령/요구하기', '턴토크 사인(관습적 반응)',
       '(언약) 약속하기(제3자와)/(개인적 수준)', '(표현) 인사하기', '(선언/위임하기)']

# 필요 문장 구성하는 모든 token 바구니에 넣기
for intenti in range(len(M_intent)):
    if df.loc[(df['intent'] == intent), 'sentence'].count() > 1000:
        for i in (df.loc[(df['intent'] == intent), 'sentence']):
            M_count += tokenizer.encode(i)
    else:
        for i in (df.loc[(df['intent'] == intent), 'sentence']):
            M_count += tokenizer.encode(i)

# token 고유값과 개수 / N_uni[0] = 고유값 리스트 / N_uni[1] = 고유값 개수 리스트
M_uni = np.unique(M_count, return_counts=True)

# 고유값 개수 내림차순 정렬
M_cnt = sorted(list(set(M_uni[1])),reverse=True)

# M_many = N_uni[0]에서 검색할 index값 / M_token = 사용 횟수가 많은 token
M_many = []
M_token = []

for i in M_cnt:
    if i > 100000 :
        M_many.append(list(M_uni[1]).index(i))
for j in range(len(M_many)) :
    M_token.append(M_uni[0][M_many[j]])
print(M_many)

tokenizer.decode(M_token)

[1, 841, 11838, 13, 1311, 1053, 1473, 1274, 1340, 1275, 1310, 1389, 1339]


'[CLS] 응 맞아.키 키는어도이고에지'

In [ ]:
# 중복값 제거
print(tokenizer.decode((set(N_token).difference(set(P_token))).difference(set(M_token))))
print(tokenizer.decode((set(P_token).difference(set(N_token))).difference(set(M_token))))

안? 아니
응응 있 나


In [ ]:
N_keyword = ['싫어\.\.', '아니']
P_keyword = ['맞아', '응응', '응\.', '응,']

In [ ]:
# 부정키워드 포함 긍정문장 제거
for intent in P_intent:
    for key in N_keyword:
        df.drop(df.loc[(df['intent'] == intent) & df['sentence'].str.contains(key)].index, axis=0, inplace=True)

# 긍정키워드 포함 부정문장 제거
for intent in N_intent:
    for key in P_keyword:
        df.drop(df.loc[(df['intent'] == intent) & df['sentence'].str.contains(key)].index, axis=0, inplace=True)

# 필요한 데이터를 추출하기 위한 DataFrame 생성
temp_df = pd.DataFrame()
temp = pd.DataFrame()
df_ = copy.deepcopy(df)

# 기존 데이터에서 긍정, 부정 문장 제거
for intent in (N_intent+P_intent):
    df_.drop(df_[df_['intent'] == intent].index, axis=0, inplace=True)

# 남은 중립문장 중 필요한만큼 문장 추출
temp_df = pd.concat([temp_df,df_.sample(n=130000)], ignore_index=True)

# 중립문장 중 긍정/부정 키워드 포함한 문장 제거
for key in (N_keyword+P_keyword):
    temp_df.drop(temp_df.loc[temp_df['sentence'].str.contains(key)].index, axis=0, inplace=True)

In [ ]:
# 필요한만큼 긍정/부정문장 추출
for intent in (N_intent+P_intent):
    print(intent)
    if df.loc[df['intent']==intent, 'intent'].count() > 100000:
        temp = pd.concat([temp,df[df['intent'] == intent].sample(n=100000)], ignore_index=True)
    # elif df.loc[df['intent']==intent, 'intent'].count() > 10000:
    #     temp = pd.concat([temp,df[df['intent'] == intent].sample(n=10000)], ignore_index=True)
    else :
        temp = pd.concat([temp,df[df['intent'] == intent]], ignore_index=True)

temp = pd.concat([temp,temp_df], ignore_index=True)

print('='*100)
for i in list(temp['intent'].unique()):
  print(i)

print(temp)

temp.loc[(temp['intent'] == '(표현) 부정감정 표현하기')|(temp['intent'] == '(언약) 거절하기')|
         (temp['intent'] == '(단언) 반박하기'), 'intent'] = '부정'
temp.loc[(temp['intent'] == '(표현) 긍정감정 표현하기')|(temp['intent'] == '(표현) 감사하기'), 'intent'] = '긍정'
temp.loc[(temp['intent'] != '부정')&(temp['intent'] != '긍정'), 'intent'] = '중립'

(표현) 부정감정 표현하기
(단언) 반박하기
(언약) 거절하기
부정
(표현) 긍정감정 표현하기
(표현) 감사하기
긍정
(표현) 부정감정 표현하기
(단언) 반박하기
(언약) 거절하기
부정
(표현) 긍정감정 표현하기
(표현) 감사하기
긍정
(지시) 부탁하기
(단언) 진술하기
(단언) 주장하기
(언약) 약속하기(제3자와)/(개인적 수준)
(지시) 충고/제안하기
(지시) 질문하기
턴토크 사인(관습적 반응)
(표현) 인사하기
(지시) 명령/요구하기
(선언/위임하기)
(표현) 사과하기
                                        sentence          intent
0                       그러니까 키키 하 진짜 너무 한 거 아니냐고  (표현) 부정감정 표현하기
1                            자본주의 ㅠ 넘 슬프네요 진짜...  (표현) 부정감정 표현하기
2                            그런 아저씨를 데리고 가시다니 ㅠㅠ  (표현) 부정감정 표현하기
3                 서울에만 1천 명대가 나왔다는데... 너무 무섭네...  (표현) 부정감정 표현하기
4                 음식으로도 많이 장난치던데 그게 제일 화나는 거 같다!  (표현) 부정감정 표현하기
...                                          ...             ...
228748                               그건 히어로 영화지?       (지시) 질문하기
228749                     키키 역시 우린 쥬니어네이버 세대 키키       (단언) 주장하기
228750                     버스에서 전화하는 아저씨들... 무서워       (단언) 주장하기
228751  나중에 6주 만에 맞은 사람만 부스터샷 맞으라고 하면 나 진짜 욕할 거야       (단언) 주장하기
228752           

In [ ]:
fin_df = copy.deepcopy(temp)

In [ ]:
temp = list(fin_df['intent'].unique())
for i in temp :
    print(i,fin_df.loc[fin_df['intent']==i,'intent'].count())

부정 27522
긍정 75827
중립 125404


In [ ]:
# 클래수 개수
lable_num = len(fin_df['intent'].unique())
lable_num

3

In [ ]:
label_dict = {0: '긍정', 1: '부정', 2: '중립'}

for idx, intent_lab in label_dict.items() :
    fin_df.loc[fin_df['intent'] == intent_lab, 'intent'] = idx

print(label_dict)
fin_df = fin_df.sample(frac=1).reset_index(drop=True)
fin_df

{0: '긍정', 1: '부정', 2: '중립'}


,sentence,intent
0,키키 아 웃으면 안되는데 개 웃기네 ㅠㅠ,2
1,디저트는 역시 케잌,2
2,난 고슴도치봄,2
3,저는 1차때가 더 힘들었어요. 그래도 부장님이 죽 사먹으라고 기프티콘도 보내주시고 ...,0
4,그럼 피아노 학원 다녀보는 건 어때?,2
...,...,...
228748,"아, 맞네 키키 나도 그거 시즌 1부터 다시 보기나 할까",2
228749,진주 안와도 산청은 옴 ㅋ,2
228750,응 키키 가서 낚시나 수영을 즐길 수 있다고 하더라,0
228751,글램핑 하고 싶다,2


In [ ]:
train_data = pd.DataFrame()

len_ = fin_df['intent'].count()

train_data = pd.concat([train_data,fin_df.loc[:(len_*8)/10]],ignore_index=True)
print(train_data)

                                                 sentence intent
0                                  키키 아 웃으면 안되는데 개 웃기네 ㅠㅠ      2
1                                              디저트는 역시 케잌      2
2                                                 난 고슴도치봄      2
3       저는 1차때가 더 힘들었어요. 그래도 부장님이 죽 사먹으라고 기프티콘도 보내주시고 ...      0
4                                    그럼 피아노 학원 다녀보는 건 어때?      2
...                                                   ...    ...
182998                                   키키 엥 진짜? 왜 안 했어?      2
182999                                그쵸 태연이 모델을 해서 그런가봐요      2
183000                           맞아, 나도 그런데 생각해 본 적 있긴 해.      0
183001                                    맞어 이번주 내가 빌려줄까?      0
183002                                             아니 검정색      1

[183003 rows x 2 columns]


In [ ]:
test_data = pd.DataFrame()

len_ = fin_df['intent'].count()

test_data = pd.concat([test_data,fin_df.loc[(len_*8)/10:]],ignore_index=True)
print(test_data)

                                     sentence intent
0      아니 언니랑 공짜 영화 티켓 생겨 가지고 영화관에 영화 보러 왔거든?      1
1                                 진해에는 뭐가 있어?      2
2                 게임 시작한지 1분도 안돼서 죽는 사람보다는 낫다      2
3                                  맞어 그맛에 갔는데      0
4                         서울에서 꽤 가까워서 가는길이 편해      2
...                                       ...    ...
45745         아, 맞네 키키 나도 그거 시즌 1부터 다시 보기나 할까      2
45746                          진주 안와도 산청은 옴 ㅋ      2
45747            응 키키 가서 낚시나 수영을 즐길 수 있다고 하더라      0
45748                               글램핑 하고 싶다      2
45749                                  웅 베리든든      0

[45750 rows x 2 columns]


In [ ]:
max_seq_len = max(df['sentence'].apply(lambda x: len(str(x))))

sent1 = train_data['sentence'].iloc[0]
print(max_seq_len)
print('문 장1 :',sent1)

100
문 장1 : 키키 아 웃으면 안되는데 개 웃기네 ㅠㅠ


In [ ]:
# 정수인코딩
encoded_result = tokenizer.encode(sent1,max_length=max_seq_len, pad_to_max_length=True)
print(encoded_result)
print('길 이 :', len(encoded_result))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[2, 1754, 2089, 1376, 1474, 6076, 1378, 2496, 13964, 558, 12494, 2203, 6516, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
길 이 : 100


In [ ]:
# 세그멘트인코딩
print([0]*max_seq_len)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
# 어텐션마스크인코딩
valid_num = len(tokenizer.encode(sent1))
print(valid_num * [1] + (max_seq_len - valid_num) * [0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        # input_id는 워드임베딩을 위한 문장의 정수인코딩
        input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True)

        # attention_mask는 실제단어가 위치하면 1, 패딩의 위치에는 0인 시퀀스.
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count

        # token_type_id은 세그먼트인코딩
        token_type_id = [0] * max_seq_len

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention masklength {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length{} vs {}".format(len(token_type_id), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [ ]:
train_X, train_y = convert_examples_to_features(train_data['sentence'], train_data['intent'],
                                              max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 183003/183003 [00:35<00:00, 5227.09it/s]


In [ ]:
test_X, test_y = convert_examples_to_features(test_data['sentence'], test_data['intent'],
                                              max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 45750/45750 [00:08<00:00, 5107.95it/s]


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=lable_num, from_pt=True)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.fit(train_X, train_y, epochs=4, batch_size=128, validation_split=0.2)

Epoch 1/4
1144/1144 [==============================] - 365s 280ms/step - loss: 0.1897 - accuracy: 0.9366 - val_loss: 0.1679 - val_accuracy: 0.9431
Epoch 2/4
1144/1144 [==============================] - 308s 269ms/step - loss: 0.1533 - accuracy: 0.9454 - val_loss: 0.1702 - val_accuracy: 0.9446
Epoch 3/4
1144/1144 [==============================] - 308s 269ms/step - loss: 0.1228 - accuracy: 0.9527 - val_loss: 0.1818 - val_accuracy: 0.9390
Epoch 4/4
1144/1144 [==============================] - 308s 269ms/step - loss: 0.0820 - accuracy: 0.9679 - val_loss: 0.2252 - val_accuracy: 0.9286


In [ ]:
results = model.evaluate(test_X, test_y, batch_size=64)
print("test loss, test acc: ", results)

715/715 [==============================] - 38s 53ms/step - loss: 0.2238 - accuracy: 0.9285
test loss, test acc:  [0.22375456988811493, 0.9285027384757996]


In [ ]:
def sentiment_predict(new_sentence):
    input_id = tokenizer.encode(new_sentence, max_length=max_seq_len, pad_to_max_length=True)

    padding_count = input_id.count(tokenizer.pad_token_id)
    attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
    token_type_id = [0] * max_seq_len

    input_ids = np.array([input_id])
    attention_masks = np.array([attention_mask])
    token_type_ids = np.array([token_type_id])

    encoded_input = [input_ids, attention_masks, token_type_ids]

    score = np.argmax(model.predict(encoded_input)[0])

    print('sentence :',new_sentence)
    print(label_dict[score])

In [ ]:
MODEL_SAVE_PATH = os.path.join("/content/Drive/MyDrive/NP_classifier/NP_classifier")
print(MODEL_SAVE_PATH)

if os.path.exists(MODEL_SAVE_PATH):
    print(f"{MODEL_SAVE_PATH} -- Folder already exists \n")

else:
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    print(f"{MODEL_SAVE_PATH} -- Folder create complete \n")

model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

/content/Drive/MyDrive/NP_classifier/NP_classifier
/content/Drive/MyDrive/NP_classifier/NP_classifier -- Folder create complete 



('/content/Drive/MyDrive/NP_classifier/NP_classifier/tokenizer_config.json',
 '/content/Drive/MyDrive/NP_classifier/NP_classifier/special_tokens_map.json',
 '/content/Drive/MyDrive/NP_classifier/NP_classifier/vocab.txt',
 '/content/Drive/MyDrive/NP_classifier/NP_classifier/added_tokens.json')

In [ ]:
new_sentence = input('sentence > ')
sentiment_predict(new_sentence)

sentence > 인공지능 재미있네!
1/1 [==============================] - 0s 43ms/step
sentence : 인공지능 재미있네!
긍정
